# Numerische Methoden für Differentialgleichungen

## Aufgabenblatt 3
*gelöst von Stollmayer Max*

### Aufgabe 1

Sei $y_{n+1} = y_n + h\ \Phi (t_n, y_n, h)$ eine explizite Ein-Schritt-Methode mit $\Phi (t, y, h) = a_1\ f(t,y) + a_2\ f\big(t+b_2\ h,\ y+ b_2\ h\ f(t,y)\big)$ und Koeffizienten $a_1 = \frac{1}{4},\ a_2 = \frac{1}{4},\ b_2 = \frac{2}{3}$ für das Modellproblem sei $y' = \lambda\ y$ mit $\lambda > 0$.

Die Folge $(y_n)_{n \ge 0}$ ist genau dann beschränkt, wenn $-\frac{2}{\lambda} \le h \le 0$ gilt.

#### Beweis
Für $y' = \lambda\ y$ ist $f(t,y) = f(y) = \lambda\ y$. Damit lässt sich $y_{n+1}$ explizit hinschreiben:

\begin{align}
y_{n+1} &= y_n + h\ \bigg(\frac{1}{4}\ f(y_n) + \frac{1}{4}\ f\big(y_n + \frac{2}{3}\ h\ f(y_n)\big)\bigg) \\
&= y_n + \frac{\lambda}{4}\ h\ y_n + \frac{3\ \lambda}{4}\ h\ \big(y_n + \frac{2\ \lambda}{3}\ h\ y_n\big) \\
&= \big(1 + \frac{\lambda}{4}\ h + \frac{3\ \lambda}{4}\ h + \frac{\lambda^2}{2}\ h^2\big)\ y_n \\
&= \big(1 + \lambda\ h + \frac{\lambda^2}{2}\ h^2\big)\ y_n \\
&=\ ... \\
&= \big(1 + \lambda\ h + \frac{\lambda^2}{2}\ h^2\big)^{n+1}\ y_0
\end{align}

Die geometrische Folge $(y_n)_{n \ge 0}$ ist nun genau dann beschränkt, wenn $|1 + \lambda\ h + \frac{\lambda^2}{2}\ h^2| \le 1$ gilt, also äquivalent, wenn gilt $-2 \le \lambda\ h + \frac{\lambda^2}{2}\ h^2 \le 0$.

$\bullet \quad$ $\frac{\lambda^2}{2}\ h^2 + \lambda\ h + 2 > 0 $ gilt für jedes $h \in \mathbb{R}$ und $\lambda > 0$.

$\bullet \quad$ $\frac{\lambda^2}{2}\ h^2 + \lambda\ h \le 0 \iff h\ \big(\frac{\lambda}{2}\ h + 1\big) \le 0 \iff \begin{cases}h \le 0 \land \frac{\lambda}{2}\ h + 1 \ge 0 \\ h \ge 0 \land \frac{\lambda}{2}\ h + 1 \le 0 \end{cases} \iff h \le 0 \land h \ge -\frac{2}{\lambda}$

Da für $h \ge 0$ die Ungleichung $\frac{\lambda}{2}\ h + 1 \le 0$ nicht gelten kann, ist nach einer kurzen Umformung die Behauptung $-\frac{2}{\lambda} \le h \le 0$ gezeigt.

### Aufgabe 2

Zu implementieren ist das Crank-Nicholson-Verfahren mit der Fixpunkt-Iteration:

\begin{cases}
y_{n+1}^{(k+1)} = y_n + \frac{h}{2} \ \big(f(t_n, y_n) + f(t_{n+1},\ y_{n+1}^{(k)})\big), \quad k \ge 0 \\
y_{n+1}^{(0)} = y_n
\end{cases}

und das implizite Euler-Verfahren mit der Fixpunkt-Iteration:

\begin{cases}
y_{n+1}^{(k+1)} = y_n + h \ f(t_{n+1},\ y_{n+1}^{(k)}), \quad k \ge 0 \\
y_{n+1}^{(0)} = y_n
\end{cases}